In [ ]:
import numpy as np
from sklearn import datasets
from matplotlib import pyplot as plt
import cv2
from skimage.transform import resize
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import h5py
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/Colab_Notebooks/'
path2 = '/content/drive/My Drive/Colab Notebooks/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Transform used to process training set and testing set data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

# Create training set
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

# Create training loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

# Create testing set
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

# Create testing loaders
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

# All possible digit classes
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [ ]:
# Net class with the specified layers
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(64 * 64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 64 * 64)
        x = self.fc1(x)
        return x

# Instantiate net class
net = Net()

In [ ]:
# Set loss function
criterion = nn.CrossEntropyLoss()

# Instantiate SGD optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [ ]:
# Train over 3 epochs
for epoch in range(3):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 500 == 499:    # print every 500 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0

print('Finished Training')

[1,   500] loss: 2.291
[1,  1000] loss: 2.240
[1,  1500] loss: 1.735
Finished Training


In [ ]:
# Save our net as mnist_net.pt.pth
PATH = './mnist_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
def split_image(image):
  preprocessed_digits = []
  image = np.squeeze(image)
  grey = image.copy()
  thresh = grey.copy()
  contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  for c in contours:
      x,y,w,h = cv2.boundingRect(c)

      cv2.rectangle(image, (x,y), (x+w, y+h), color=(0, 255, 0), thickness=2)

      digit = thresh[y:y+h, x:x+w]
      
      resized_digit = cv2.resize(digit, (18,18))
      
      padded_digit = np.pad(resized_digit, ((7,7),(7,7)), "constant", constant_values=0)
      
      preprocessed_digits.append(padded_digit)

  digits = torch.from_numpy(np.array(preprocessed_digits, dtype='uint8'))
  return digits

In [ ]:
def sum_left_col(window):
  left_col = window[:,:1,:]
  sum_left_col = left_col.sum(axis=1)
  return np.max(sum_left_col) == 0

In [ ]:
def kernel_blur(image,w,h):
  kernel = np.ones((3,3),np.float32)/9
  dst = cv2.filter2D(image,-1,kernel)
  dst = np.reshape(dst, (w,h,1))
  return dst

In [ ]:
def split_image2(image):
  digits2 = []
  img = image.copy()

  img = kernel_blur(img,64,64)

  idx = img[:,:,0] < 40
  img[idx,0] = 0

  idx2 = img[:,:,0] > 90
  img[idx2,0] = 255
  
  '''
  test = np.squeeze(img)
  plt.imshow(test, cmap='gray')
  plt.show()
  '''
  prev = None
  # Assuming square image
  for x in range(len(img)):
    window = img[24:40,x:x+10,:]
    is_left_black = sum_left_col(window)
    
    if prev and not is_left_black:
      if x + 10 > len(img) - 1:
        digits2.append(image[24:40,x:,:])
        break
      else:
        digits2.append(image[24:40,x:x+10,:])

    prev = is_left_black
  
  return digits2

In [ ]:
hf = h5py.File(path + 'MNIST_synthetic.h5', 'r')
print(hf.keys())

X_train = hf['train_dataset']
print(X_train.shape)

y_train = np.array(hf['train_labels'])
print(y_train.shape)

X_test = hf['test_dataset']
print(X_test.shape)

y_test = np.array([np.array([9,10,10,10,10]),np.array([0,1,7,3,10]),np.array([1,4,0,8,3]),np.array([0,10,10,10,10]),np.array([4,6,7,8,10]),np.array([0,10,10,10,10]),np.array([6,9,2,10,10]),np.array([3,7,10,10,10]),np.array([4,8,2,6,7]),np.array([9,6,6,0,10])])
print(y_test.shape)

<KeysViewHDF5 ['test_dataset', 'train_dataset', 'train_labels']>
(56000, 64, 64, 1)
(56000, 5)
(14000, 64, 64, 1)
(10, 5)


In [ ]:
# Instantiate net class
net = Net()
net.load_state_dict(torch.load(PATH))

test_im3 = []
probs3 = []
for i in range(len(X_train)):
  temp3 = X_train[i].copy()
  digits3 = split_image2(temp3)

  if 5-(list(y_train[i]).count(10)) != len(digits3):
    probs3.append(i)
  for j3 in digits3:
    test_im3.append(j3)
      
print('Total number of digits detected: ' + str(len(test_im3)))
print('Total number of problematic images: ' + str(len(probs3)))


Total number of digits detected: 167893
Total number of problematic images: 41


In [ ]:
# Displays the problematic images identified above in probs3
for i in range(len(probs3)):
  im3 = np.squeeze(X_train[probs3[i]])
  plt.imshow(im3, cmap='gray')
  plt.show()
  digits3 = split_image2(X_train[probs3[i]])

In [ ]:
c = 0
for y in y_train:
  c += 5 - list(y).count(10)

print('There are ' + str(c) + ' digits in the ' + str(len(y_train)) + ' training sample images')

There are 167886 digits in the 56000 training sample images


In [ ]:
# Rayan's Additions here!
class H5Dataset(torch.utils.data.Dataset):

  def __init__(self, h5File):
      super(H5Dataset, self).__init__()
      hf = h5py.File(h5File , 'r')
      self.train_label = np.array(hf['train_labels'])
      self.train_label = self.train_label.flatten()
      self.train_label = self.train_label[self.train_label != 10]
      self.train_data = np.array(hf['train_dataset'])
      test_im = []
      for i in range(len(self.train_data)):
        for j in split_image(self.train_data[i]):
          test_im.append(j)
      self.train_data = torch.stack(test_im) 
      self.train_data = torch.unsqueeze(self.train_data, 1)


  def __getitem__(self, index):
    return (self.train_data[index, :,:, :], torch.from_numpy(self.train_label)[index])

  def __len__(self):
      return self.train_label.shape[0]

In [ ]:
train_dataset = H5Dataset(path2 + 'MNIST_synthetic.h5')
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle = True, num_workers=2)


dataiter = iter(trainloader)
images,labels = dataiter.next()

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        #print(images.shape)
        #print(labels.shape)
        _, predicted = torch.max(outputs.data, 1)
        #print(predicted.shape)
        if (len(predicted) != len(labels)):
          predicted = predicted[0:16]
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

# END OF RAYAN'S ADDITIONS!